In [19]:
import pandas as pd
import numpy as np
other_out = pd.read_csv('../data/crc_raw/CRC_2.0-public_clinical_data/regimen_cancer_level_dataset.csv')
other_out['drugs_combined'] = other_out[['drugs_drug_1', 'drugs_drug_2', 'drugs_drug_3', 'drugs_drug_4', 'drugs_drug_5']].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)
other_out['drugs_combined'].value_counts()
data_ib = pd.read_csv('../data/crc_raw/cBioPortal_files/data_timeline_treatment.txt', sep='\t')
reg_df = data_ib['REGIMEN'].value_counts()
reg_df.head(20)

REGIMEN
Everolimus, Fluorouracil, Leucovorin Calcium, Oxaliplatin    4
Everolimus, Investigational Drug                             2
Name: count, dtype: int64

In [5]:
data_ib = data_ib.loc[(data_ib['REGIMEN'].str.contains('cetux', case=False, na=False)) | (data_ib['REGIMEN'].str.contains('panitu', case=False, na=False))]
data_ib['REGIMEN'].value_counts()


REGIMEN
Cetuximab, Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium                 316
Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium, Panitumumab               224
Irinotecan Hydrochloride, Panitumumab                                                 148
Cetuximab, Irinotecan Hydrochloride                                                   142
Fluorouracil, Leucovorin Calcium, Oxaliplatin, Panitumumab                             80
Cetuximab, Fluorouracil, Leucovorin Calcium, Oxaliplatin                               72
Bevacizumab, Cetuximab, Irinotecan Hydrochloride                                       36
Panitumumab                                                                            36
Dabrafenib, Panitumumab, Trametinib                                                    24
Investigational Drug, Irinotecan Hydrochloride, Panitumumab                            21
Cetuximab, Fluorouracil, Leucovorin Calcium                                            18
Ce

In [6]:
other_out.head()

,cohort,record_id,institution,ca_seq,regimen_number,regimen_number_within_cancer,redcap_ca_index,drugs_num,drugs_inst,drugs_firstinst,...,ttnt_any_ca_status,ttnt_any_ca_days,ttnt_any_ca_mos,ttnt_any_ca_yrs,ttnt_ca_seq_status,ttnt_ca_seq_days,ttnt_ca_seq_mos,ttnt_ca_seq_yrs,release_version,drugs_combined
0,CRC,GENIE-DFCI-000048,DFCI,0,1,1,Yes,2,At the internal/native institution only,NaN,...,0,273,8.980263,0.747433,0,273,8.980263,0.747433,2.0-public,"Fluorouracil(5FU,5Fluorouracil,5Fluracil,AccuS..."
1,CRC,GENIE-DFCI-000147,DFCI,0,1,1,Yes,1,At external institution only,NaN,...,1,181,5.953947,0.495551,1,181,5.953947,0.495551,2.0-public,"Capecitabine(Ro091978 000,Xeloda)"
2,CRC,GENIE-DFCI-000147,DFCI,0,2,2,Yes,2,At external institution only,NaN,...,1,275,9.046053,0.752909,1,275,9.046053,0.752909,2.0-public,"Capecitabine(Ro091978 000,Xeloda),Oxaliplatin(..."
3,CRC,GENIE-DFCI-000147,DFCI,0,3,3,Yes,4,At external institution only,NaN,...,1,473,15.559211,1.295003,1,473,15.559211,1.295003,2.0-public,"Fluorouracil(5FU,5Fluorouracil,5Fluracil,AccuS..."
4,CRC,GENIE-DFCI-000147,DFCI,0,4,4,Yes,1,At the internal/native institution only,NaN,...,1,282,9.276316,0.772074,1,282,9.276316,0.772074,2.0-public,Trifluridine and Tipiracil Hydrochloride(Lonsu...


In [4]:
ib_regimens = data_ib['REGIMEN'].unique()
len(ib_regimens)
ib_list = [reg.split(',') for reg in ib_regimens]
ib_list
#if 'Investigational Drug' is one of the drugs, drop it from the list
for i in ib_list:
    if 'Investigational Drug' in i:
        i.remove('Investigational Drug')
    elif ' Investigational Drug' in i:
        i.remove(' Investigational Drug')
#strip leading and trailing spaces
ib_list = [[drug.strip() for drug in reg] for reg in ib_list]
ib_list
treatment_list = data_ib['REGIMEN'].unique()
#save to csv
# pd.DataFrame(treatment_list).to_csv('../data/egfr_list.csv', index=False)
#find the rows in other_out that contain each of the drugs in each element of ib_list
ib_out = pd.DataFrame()
for i in ib_list:
    if len(i) == 1:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False)]
        #make a new column with the list of drugs in each row
        new_df['drugs_list'] = str(i[0])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 2:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 3:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False) & other_out['drugs_combined'].str.contains(i[2], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1]) + ', ' + str(i[2])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 4:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False) & other_out['drugs_combined'].str.contains(i[2], case=False, na=False) & other_out['drugs_combined'].str.contains(i[3], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1]) + ', ' + str(i[2]) + ', ' + str(i[3])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
ib_out.shape
ib_out.head()
cols = ib_out.columns.to_list()
ib_out['pfs_i_and_m_g_status'].value_counts()
keep = ['record_id', 'institution', 'drugs_num', 'drugs_list', 'len_reg', 'dx_drug_start_int_1', 'os_g_status', 'pfs_i_and_m_g_status']
clin = ib_out.copy()
ib_out = ib_out[keep]
ib_out.sort_values(by=['record_id', 'drugs_num', 'len_reg'], inplace=True)
ib_out.head()

print(ib_out.shape)
ib_out = ib_out.drop_duplicates(subset=['record_id', 'drugs_num', 'drugs_list'], keep='first')
print(ib_out.shape)
print(ib_out.shape)
ib_out = ib_out.drop_duplicates(subset=['record_id', 'drugs_num'], keep='last')
print(ib_out.shape)
ib_out['drugs_list'].value_counts()
ib_out.reset_index(inplace=True, drop=True)
ib_out['sample_id'] = np.NaN
ib_out['diff'] = np.NaN
panel = pd.read_csv('../data/crc_raw/CRC_2.0-public_clinical_data/cancer_panel_test_level_dataset.csv', index_col='cpt_genie_sample_id')
keep = ['record_id', 'dx_cpt_rep_days']
panel = panel[keep]
panel.sort_values(by=['record_id'], inplace=True)

for i in range(len(ib_out)):
    
    record_id = ib_out.iloc[i]['record_id']
    panel_rows = panel.loc[panel['record_id'] == record_id]
    panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
    if len(panel_rows) == 1:
        ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
        ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
    elif len(panel_rows) > 1:
        #use the row with the smallest difference, as long as it's less than 0, unless they are all positive
        if len(panel_rows.loc[panel_rows['diff'] < 0]) == 0:
            panel_rows.sort_values(by=['diff'], inplace=True)
            ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
            ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
        else:
            panel_rows = panel_rows.loc[panel_rows['diff'] < 0]
            panel_rows.sort_values(by=['diff'], inplace=True)
            ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
            ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
ib_out.head()
ib_out.drop(['drugs_num', 'len_reg', 'dx_drug_start_int_1', 'diff'], axis=1, inplace=True)
ib_out.head()


/tmp/ipykernel_14004/3692510741.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1]) + ', ' + str(i[2])
/tmp/ipykernel_14004/3692510741.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['len_reg'] = len(i)
/tmp/ipykernel_14004/3692510741.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

(2232, 8)
(1269, 8)
(1269, 8)
(368, 8)


/tmp/ipykernel_14004/3692510741.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
/tmp/ipykernel_14004/3692510741.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
/tmp/ipykernel_14004/3692510741.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

,record_id,institution,drugs_list,os_g_status,pfs_i_and_m_g_status,sample_id
0,GENIE-DFCI-000971,DFCI,"Bevacizumab, Cetuximab, Irinotecan Hydrochloride",1,1.0,GENIE-DFCI-000971-10958
1,GENIE-DFCI-001292,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin,...",1,1.0,GENIE-DFCI-001292-7406
2,GENIE-DFCI-001463,DFCI,Cetuximab,1,0.0,GENIE-DFCI-001463-9650
3,GENIE-DFCI-001463,DFCI,"Cetuximab, Irinotecan Hydrochloride",1,1.0,GENIE-DFCI-001463-9650
4,GENIE-DFCI-002507,DFCI,"Fluorouracil, Irinotecan Hydrochloride, Leucov...",1,1.0,GENIE-DFCI-002507-4815


In [5]:

ib_out['pfs_i_and_m_g_status'].value_counts()


pfs_i_and_m_g_status
1.0    191
0.0    171
Name: count, dtype: int64

In [6]:
ib_out['os_g_status'].value_counts()


os_g_status
1    241
0    127
Name: count, dtype: int64

In [7]:
ib_out.to_csv('../data/crc_egfr_out.csv', index=True)

In [8]:
keep2 = ['record_id', 'institution', 'drugs_num', 'drugs_list', 'len_reg', 'dx_drug_start_int_1', 'os_g_status', 'pfs_i_and_m_g_status', 'tt_os_dx_days']
clin = ib_out.copy()
ib_out = ib_out[keep2]
ib_out.sort_values(by=['record_id', 'drugs_num', 'len_reg'], inplace=True)
ib_out.head()

print(ib_out.shape)
ib_out = ib_out.drop_duplicates(subset=['record_id', 'drugs_num', 'drugs_list'], keep='first')
print(ib_out.shape)
print(ib_out.shape)
ib_out = ib_out.drop_duplicates(subset=['record_id', 'drugs_num'], keep='last')
print(ib_out.shape)
ib_out['drugs_list'].value_counts()
ib_out.reset_index(inplace=True, drop=True)
ib_out['sample_id'] = np.NaN
ib_out['diff'] = np.NaN
panel = pd.read_csv('../data/crc_raw/CRC_2.0-public_clinical_data/cancer_panel_test_level_dataset.csv', index_col='cpt_genie_sample_id')
keep = ['record_id', 'dx_cpt_rep_days']
panel = panel[keep]
panel.sort_values(by=['record_id'], inplace=True)

for i in range(len(ib_out)):
    
    record_id = ib_out.iloc[i]['record_id']
    panel_rows = panel.loc[panel['record_id'] == record_id]
    panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
    if len(panel_rows) == 1:
        ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
        ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
    elif len(panel_rows) > 1:
        #use the row with the smallest difference, as long as it's less than 0, unless they are all positive
        if len(panel_rows.loc[panel_rows['diff'] < 0]) == 0:
            panel_rows.sort_values(by=['diff'], inplace=True)
            ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
            ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
        else:
            panel_rows = panel_rows.loc[panel_rows['diff'] < 0]
            panel_rows.sort_values(by=['diff'], inplace=True)
            ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
            ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
ib_out.head()
ib_out.drop(['drugs_num', 'len_reg', 'dx_drug_start_int_1', 'diff'], axis=1, inplace=True)
ib_out.to_csv('../data/crc_egfr_surv.csv', index=True)

KeyError: "['drugs_num', 'len_reg', 'dx_drug_start_int_1', 'tt_os_dx_days'] not in index"

In [ ]:
import pandas as pd
import numpy as np
other_out = pd.read_csv('../data/crc_raw/CRC_2.0-public_clinical_data/regimen_cancer_level_dataset.csv')
other_out['drugs_combined'] = other_out[['drugs_drug_1', 'drugs_drug_2', 'drugs_drug_3', 'drugs_drug_4', 'drugs_drug_5']].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)
other_out['drugs_combined'].value_counts()
data_ib = pd.read_csv('../data/crc_raw/cBioPortal_files/data_timeline_treatment.txt', sep='\t')
data_ib = data_ib.loc[(data_ib['REGIMEN'].str.contains('bevacizumab', case=False, na=False))]
data_ib['REGIMEN'].value_counts()


REGIMEN
Bevacizumab, Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium                          2488
Bevacizumab, Fluorouracil, Leucovorin Calcium, Oxaliplatin                                       1880
Bevacizumab, Fluorouracil, Leucovorin Calcium                                                     282
Bevacizumab, Capecitabine                                                                         168
Bevacizumab, Irinotecan Hydrochloride                                                             130
Bevacizumab, Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium, Oxaliplatin              125
Bevacizumab, Capecitabine, Oxaliplatin                                                             96
Bevacizumab, Fluorouracil, Irinotecan Hydrochloride                                                45
Bevacizumab, Capecitabine, Irinotecan Hydrochloride                                                45
Bevacizumab, Cetuximab, Irinotecan Hydrochloride                          

In [ ]:

ib_regimens = data_ib['REGIMEN'].unique()
len(ib_regimens)
ib_list = [reg.split(',') for reg in ib_regimens]
ib_list
#if 'Investigational Drug' is one of the drugs, drop it from the list
for i in ib_list:
    if 'Investigational Drug' in i:
        i.remove('Investigational Drug')
    elif ' Investigational Drug' in i:
        i.remove(' Investigational Drug')
#strip leading and trailing spaces
ib_list = [[drug.strip() for drug in reg] for reg in ib_list]
ib_list
treatment_list = data_ib['REGIMEN'].unique()


#find the rows in other_out that contain each of the drugs in each element of ib_list
ib_out = pd.DataFrame()
for i in ib_list:
    if len(i) == 1:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False)]
        #make a new column with the list of drugs in each row
        new_df['drugs_list'] = str(i[0])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 2:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 3:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False) & other_out['drugs_combined'].str.contains(i[2], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1]) + ', ' + str(i[2])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 4:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False) & other_out['drugs_combined'].str.contains(i[2], case=False, na=False) & other_out['drugs_combined'].str.contains(i[3], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1]) + ', ' + str(i[2]) + ', ' + str(i[3])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
ib_out.shape
ib_out.head()
cols = ib_out.columns.to_list()
ib_out['pfs_i_and_m_g_status'].value_counts()
keep = ['record_id', 'institution', 'drugs_num', 'drugs_list', 'len_reg', 'dx_drug_start_int_1', 'os_g_status', 'pfs_i_and_m_g_status']
clin = ib_out.copy()
ib_out = ib_out[keep]
ib_out.sort_values(by=['record_id', 'drugs_num', 'len_reg'], inplace=True)
ib_out.head()

print(ib_out.shape)
ib_out = ib_out.drop_duplicates(subset=['record_id', 'drugs_num', 'drugs_list'], keep='first')
print(ib_out.shape)
print(ib_out.shape)
ib_out = ib_out.drop_duplicates(subset=['record_id', 'drugs_num'], keep='last')
print(ib_out.shape)
ib_out['drugs_list'].value_counts()
ib_out.reset_index(inplace=True, drop=True)
ib_out['sample_id'] = np.NaN
ib_out['diff'] = np.NaN
panel = pd.read_csv('../data/crc_raw/CRC_2.0-public_clinical_data/cancer_panel_test_level_dataset.csv', index_col='cpt_genie_sample_id')
keep = ['record_id', 'dx_cpt_rep_days']
panel = panel[keep]
panel.sort_values(by=['record_id'], inplace=True)

for i in range(len(ib_out)):
    
    record_id = ib_out.iloc[i]['record_id']
    panel_rows = panel.loc[panel['record_id'] == record_id]
    panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
    if len(panel_rows) == 1:
        ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
        ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
    elif len(panel_rows) > 1:
        #use the row with the smallest difference, as long as it's less than 0, unless they are all positive
        if len(panel_rows.loc[panel_rows['diff'] < 0]) == 0:
            panel_rows.sort_values(by=['diff'], inplace=True)
            ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
            ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
        else:
            panel_rows = panel_rows.loc[panel_rows['diff'] < 0]
            panel_rows.sort_values(by=['diff'], inplace=True)
            ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
            ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
ib_out.head()
ib_out.drop(['drugs_num', 'len_reg', 'dx_drug_start_int_1', 'diff'], axis=1, inplace=True)
ib_out.head()


/tmp/ipykernel_509764/4090412069.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1]) + ', ' + str(i[2]) + ', ' + str(i[3])
/tmp/ipykernel_509764/4090412069.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['len_reg'] = len(i)
/tmp/ipykernel_509764/4090412069.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

(11781, 8)
(6822, 8)
(6822, 8)
(958, 8)


/tmp/ipykernel_509764/4090412069.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
/tmp/ipykernel_509764/4090412069.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
/tmp/ipykernel_509764/4090412069.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,record_id,institution,drugs_list,os_g_status,pfs_i_and_m_g_status,sample_id
0,GENIE-DFCI-000147,DFCI,"Bevacizumab, Fluorouracil, Irinotecan Hydrochl...",1,1.0,GENIE-DFCI-000147-6898
1,GENIE-DFCI-000233,DFCI,"Bevacizumab, Fluorouracil, Irinotecan Hydrochl...",1,0.0,GENIE-DFCI-000233-7633
2,GENIE-DFCI-000306,DFCI,"Bevacizumab, Fluorouracil, Leucovorin Calcium,...",1,1.0,GENIE-DFCI-000306-5914
3,GENIE-DFCI-000738,DFCI,"Bevacizumab, Fluorouracil, Irinotecan Hydrochl...",0,0.0,GENIE-DFCI-000738-11390
4,GENIE-DFCI-000946,DFCI,"Bevacizumab, Fluorouracil",1,0.0,GENIE-DFCI-000946-10743


In [ ]:
ib_out['pfs_i_and_m_g_status'].value_counts()


pfs_i_and_m_g_status
0.0    512
1.0    411
Name: count, dtype: int64

In [ ]:
ib_out['os_g_status'].value_counts()


os_g_status
1    602
0    356
Name: count, dtype: int64

In [ ]:
ib_out.to_csv('../data/crc_vegf_out.csv', index=True)

In [ ]:
keep2 = ['record_id', 'institution', 'drugs_num', 'drugs_list', 'len_reg', 'dx_drug_start_int_1', 'os_g_status', 'pfs_i_and_m_g_status', 'tt_os_dx_days']
clin = ib_out.copy()
ib_out = ib_out[keep2]
ib_out.sort_values(by=['record_id', 'drugs_num', 'len_reg'], inplace=True)
ib_out.head()

print(ib_out.shape)
ib_out = ib_out.drop_duplicates(subset=['record_id', 'drugs_num', 'drugs_list'], keep='first')
print(ib_out.shape)
print(ib_out.shape)
ib_out = ib_out.drop_duplicates(subset=['record_id', 'drugs_num'], keep='last')
print(ib_out.shape)
ib_out['drugs_list'].value_counts()
ib_out.reset_index(inplace=True, drop=True)
ib_out['sample_id'] = np.NaN
ib_out['diff'] = np.NaN
panel = pd.read_csv('../data/crc_raw/CRC_2.0-public_clinical_data/cancer_panel_test_level_dataset.csv', index_col='cpt_genie_sample_id')
keep = ['record_id', 'dx_cpt_rep_days']
panel = panel[keep]
panel.sort_values(by=['record_id'], inplace=True)

for i in range(len(ib_out)):
    
    record_id = ib_out.iloc[i]['record_id']
    panel_rows = panel.loc[panel['record_id'] == record_id]
    panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
    if len(panel_rows) == 1:
        ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
        ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
    elif len(panel_rows) > 1:
        #use the row with the smallest difference, as long as it's less than 0, unless they are all positive
        if len(panel_rows.loc[panel_rows['diff'] < 0]) == 0:
            panel_rows.sort_values(by=['diff'], inplace=True)
            ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
            ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
        else:
            panel_rows = panel_rows.loc[panel_rows['diff'] < 0]
            panel_rows.sort_values(by=['diff'], inplace=True)
            ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
            ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
ib_out.head()
ib_out.drop(['drugs_num', 'len_reg', 'dx_drug_start_int_1', 'diff'], axis=1, inplace=True)
ib_out.to_csv('../data/crc_vegf_surv.csv', index=True)

In [18]:
import pandas as pd
import numpy as np
other_out = pd.read_csv('../data/crc_raw/CRC_2.0-public_clinical_data/regimen_cancer_level_dataset.csv')
other_out['drugs_combined'] = other_out[['drugs_drug_1', 'drugs_drug_2', 'drugs_drug_3', 'drugs_drug_4', 'drugs_drug_5']].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)
other_out['drugs_combined'].value_counts()
data_ib = pd.read_csv('../data/crc_raw/cBioPortal_files/data_timeline_treatment.txt', sep='\t')
data_ib = data_ib.loc[(data_ib['REGIMEN'].str.contains('fluorouracil', case=False, na=False))]
data_ib['REGIMEN'].value_counts()


REGIMEN
Fluorouracil, Leucovorin Calcium, Oxaliplatin                                                             3054
Bevacizumab, Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium                                   2488
Bevacizumab, Fluorouracil, Leucovorin Calcium, Oxaliplatin                                                1880
Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium                                                1227
Cetuximab, Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium                                      316
Bevacizumab, Fluorouracil, Leucovorin Calcium                                                              282
Fluorouracil, Leucovorin Calcium                                                                           260
Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium, Panitumumab                                    224
Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium, Oxaliplatin                                 

In [24]:

data_ib = pd.read_csv('../data/crc_raw/cBioPortal_files/data_timeline_treatment.txt', sep='\t')
data_ib = data_ib.loc[((data_ib['REGIMEN'].str.contains('Fluorouracil', case=False, na=False) &
                        data_ib['REGIMEN'].str.contains('Oxaliplatin', case=False, na=False)) |
                        (data_ib['REGIMEN'].str.contains('Capecitabine', case=False, na=False) &
                        data_ib['REGIMEN'].str.contains('Oxaliplatin', case=False, na=False))) &
                       ~(data_ib['REGIMEN'].str.contains('mab', case=False, na=False) | 
                         data_ib['REGIMEN'].str.contains('ib', case=False, na=False) |
                         data_ib['REGIMEN'].str.contains('nib', case=False, na=False) |
                         data_ib['REGIMEN'].str.contains('investigational', case=False, na=False) |
                         data_ib['REGIMEN'].str.contains('everoli', case=False, na=False) |
                         data_ib['REGIMEN'].str.contains('iri', case=False, na=False))]
data_ib['REGIMEN'].value_counts()


REGIMEN
Fluorouracil, Leucovorin Calcium, Oxaliplatin                               3054
Capecitabine, Oxaliplatin                                                    152
Capecitabine, Fluorouracil, Leucovorin Calcium, Oxaliplatin                   16
Fluorouracil, Oxaliplatin                                                     14
Fluorouracil, Gemcitabine Hydrochloride, Leucovorin Calcium, Oxaliplatin       4
Name: count, dtype: int64

In [26]:
ib_regimens = data_ib['REGIMEN'].unique()
len(ib_regimens)
ib_list = [reg.split(',') for reg in ib_regimens]
ib_list
#if 'Investigational Drug' is one of the drugs, drop it from the list
for i in ib_list:
    if 'Investigational Drug' in i:
        i.remove('Investigational Drug')
    elif ' Investigational Drug' in i:
        i.remove(' Investigational Drug')
#strip leading and trailing spaces
ib_list = [[drug.strip() for drug in reg] for reg in ib_list]
ib_list
treatment_list = data_ib['REGIMEN'].unique()


#find the rows in other_out that contain each of the drugs in each element of ib_list
ib_out = pd.DataFrame()
for i in ib_list:
    if len(i) == 1:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False)]
        #make a new column with the list of drugs in each row
        new_df['drugs_list'] = str(i[0])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 2:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 3:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False) & other_out['drugs_combined'].str.contains(i[2], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1]) + ', ' + str(i[2])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 4:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False) & other_out['drugs_combined'].str.contains(i[2], case=False, na=False) & other_out['drugs_combined'].str.contains(i[3], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1]) + ', ' + str(i[2]) + ', ' + str(i[3])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
ib_out.shape
ib_out.head()
cols = ib_out.columns.to_list()
ib_out['pfs_i_and_m_g_status'].value_counts()
keep = ['record_id', 'institution', 'drugs_num', 'drugs_list', 'len_reg', 'dx_drug_start_int_1', 'os_g_status', 'pfs_i_and_m_g_status']
clin = ib_out.copy()
ib_out = ib_out[keep]
ib_out.sort_values(by=['record_id', 'drugs_num', 'len_reg'], inplace=True)
ib_out.head()

print(ib_out.shape)
ib_out = ib_out.drop_duplicates(subset=['record_id', 'drugs_num', 'drugs_list'], keep='first')
print(ib_out.shape)
print(ib_out.shape)
ib_out = ib_out.drop_duplicates(subset=['record_id', 'drugs_num'], keep='last')
print(ib_out.shape)
ib_out['drugs_list'].value_counts()
ib_out.reset_index(inplace=True, drop=True)
ib_out['sample_id'] = np.NaN
ib_out['diff'] = np.NaN
panel = pd.read_csv('../data/crc_raw/CRC_2.0-public_clinical_data/cancer_panel_test_level_dataset.csv', index_col='cpt_genie_sample_id')
keep = ['record_id', 'dx_cpt_rep_days']
panel = panel[keep]
panel.sort_values(by=['record_id'], inplace=True)

for i in range(len(ib_out)):
    
    record_id = ib_out.iloc[i]['record_id']
    panel_rows = panel.loc[panel['record_id'] == record_id]
    panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
    if len(panel_rows) == 1:
        ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
        ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
    elif len(panel_rows) > 1:
        #use the row with the smallest difference, as long as it's less than 0, unless they are all positive
        if len(panel_rows.loc[panel_rows['diff'] < 0]) == 0:
            panel_rows.sort_values(by=['diff'], inplace=True)
            ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
            ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
        else:
            panel_rows = panel_rows.loc[panel_rows['diff'] < 0]
            panel_rows.sort_values(by=['diff'], inplace=True)
            ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
            ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
ib_out.head()
ib_out.drop(['drugs_num', 'len_reg', 'dx_drug_start_int_1', 'diff'], axis=1, inplace=True)
ib_out.head()

print(ib_out['pfs_i_and_m_g_status'].value_counts())

print(ib_out['os_g_status'].value_counts())

ib_out.to_csv('../data/crc_folfox_out.csv', index=True)

/tmp/ipykernel_911341/3171623862.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1])
/tmp/ipykernel_911341/3171623862.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['len_reg'] = len(i)
/tmp/ipykernel_911341/3171623862.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

(3412, 8)
(2797, 8)
(2797, 8)
(1443, 8)


/tmp/ipykernel_911341/3171623862.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
/tmp/ipykernel_911341/3171623862.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
/tmp/ipykernel_911341/3171623862.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

pfs_i_and_m_g_status
0.0    740
1.0    342
Name: count, dtype: int64
os_g_status
1    756
0    687
Name: count, dtype: int64


/tmp/ipykernel_911341/3171623862.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
/tmp/ipykernel_911341/3171623862.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
/tmp/ipykernel_911341/3171623862.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer